In [1]:
import re
import pandas as pd

In [2]:
### กำหนด data type ที่เหมาะสมกับ attribute values (Custom data types) ###


def guess_column_types(file_path, delimiter=',', has_headers=True):
    try:
        # Read the CSV file using the specified delimiter and header settings
        df = pd.read_csv(file_path, sep=delimiter,low_memory=False, header=0 if has_headers else None)

        # Initialize a dictionary to store column data types
        column_types = {}

        # Loop through columns and infer data types
        for column in df.columns:
            # sample_values = df[column].dropna().sample(min(5, len(df[column])), random_state=42)

            # Check for datetime format "YYYY-MM-DD HH:MM:SS"
            is_datetime = all(re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', str(value)) for value in df[column])

            # Check for date format "YYYY-MM-DD"
            is_date = all(re.match(r'\d{4}-\d{2}-\d{2}', str(value)) for value in df[column])

            # Assign data type based on format detection
            if is_datetime:
                inferred_type = 'datetime64'
            elif is_date:
                inferred_type = 'date'
            else:
                inferred_type = pd.api.types.infer_dtype(df[column], skipna=True)

            column_types[column] = inferred_type

        return (True, column_types)  # Return success and column types
    except pd.errors.ParserError:
        return (False, str(e))  # Return error message

In [3]:
file_path = './LoanStats_web.csv'
result, column_types_or_error = guess_column_types(file_path)

if result:
    print("Column Types:", column_types_or_error)
else:
    print("Error:", column_types_or_error)


Column Types: {'id': 'string', 'member_id': 'floating', 'loan_amnt': 'floating', 'funded_amnt': 'floating', 'funded_amnt_inv': 'floating', 'term': 'string', 'int_rate': 'string', 'installment': 'floating', 'grade': 'string', 'sub_grade': 'string', 'emp_title': 'string', 'emp_length': 'string', 'home_ownership': 'string', 'annual_inc': 'floating', 'verification_status': 'string', 'issue_d': 'string', 'loan_status': 'string', 'pymnt_plan': 'string', 'url': 'floating', 'desc': 'string', 'purpose': 'string', 'title': 'string', 'zip_code': 'string', 'addr_state': 'string', 'dti': 'floating', 'delinq_2yrs': 'floating', 'earliest_cr_line': 'string', 'inq_last_6mths': 'floating', 'mths_since_last_delinq': 'floating', 'mths_since_last_record': 'floating', 'open_acc': 'floating', 'pub_rec': 'floating', 'revol_bal': 'floating', 'revol_util': 'string', 'total_acc': 'floating', 'initial_list_status': 'string', 'out_prncp': 'floating', 'out_prncp_inv': 'floating', 'total_pymnt': 'floating', 'total_p

In [4]:
# เปลี่ยน data type บางตัว ให้เหมาะสมกับ python's environment และ MSSQL
column_types_corrected = {col: ('datetime64' if t == 'date' else 'float64' if t == 'floating' else t) \
                          for col, t in column_types_or_error.items()}

In [5]:
###** นำ dictionary ที่บรรจุ ชื่อ col และ data type มาใช้เป็น parameter ของการทำ pd.read_csv ( ) อ่าน csv มาเป็น dataframe ###
raw_df = pd.read_csv(file_path, dtype=column_types_corrected)\
[['application_type', 'annual_inc', 'annual_inc_joint', 'dti', 'dti_joint', 'emp_length'\
 ,'issue_d','int_rate','home_ownership','loan_status','loan_amnt', 'funded_amnt','installment']]

In [6]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432466 entries, 0 to 1432465
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1323970 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   home_ownership    1432440 non-null  string 
 9   loan_status       1432440 non-null  string 
 10  loan_amnt         1432440 non-null  float64
 11  funded_amnt       1432440 non-null  float64
 12  installment       1432440 non-null  float64
dtypes: float64(7), string(6)
memory usage: 142.1 MB


In [7]:
##**
# ตั้งค่าการแสดงผลของ pandas ให้ไม่ใช้ scientific notation และแสดงตัวเลขเต็ม
pd.set_option('display.float_format', '{:.2f}'.format)

In [8]:
##**
raw_df.describe(include='all')

,application_type,annual_inc,annual_inc_joint,dti,dti_joint,emp_length,issue_d,int_rate,home_ownership,loan_status,loan_amnt,funded_amnt,installment
count,1432440,1432440.00,112083.00,1430856.00,112081.00,1323970,1432440,1432440,1432440,1432440,1432440.00,1432440.00,1432440.00
unique,2,NaN,NaN,NaN,NaN,11,30,258,5,7,NaN,NaN,NaN
top,Individual,NaN,NaN,NaN,NaN,10+ years,Mar-2017,11.49%,MORTGAGE,Current,NaN,NaN,NaN
freq,1320357,NaN,NaN,NaN,NaN,478304,148724,49032,701646,702223,NaN,NaN,NaN
mean,NaN,81034.58,126843.25,19.21,19.09,NaN,NaN,NaN,NaN,NaN,15370.39,15370.37,456.69
std,NaN,134183.36,77982.26,15.97,7.84,NaN,NaN,NaN,NaN,NaN,9646.03,9646.03,281.71
min,NaN,0.00,5693.51,-1.00,0.00,NaN,NaN,NaN,NaN,NaN,1000.00,1000.00,19.40
25%,NaN,48000.00,85000.00,11.99,13.35,NaN,NaN,NaN,NaN,NaN,8000.00,8000.00,250.29
50%,NaN,68000.00,113000.00,18.00,18.64,NaN,NaN,NaN,NaN,NaN,13000.00,13000.00,380.03
75%,NaN,96667.00,150000.00,24.83,24.42,NaN,NaN,NaN,NaN,NaN,20000.00,20000.00,614.54


In [9]:
#** แทนที่ค่า null ในคอลัมน์ 'emp_length' ด้วย 'N/A'
raw_df['emp_length'] = raw_df['emp_length'].fillna('N/A')

In [10]:
##**
raw_df['emp_length'].value_counts()

emp_length
10+ years    478304
2 years      130251
< 1 year     118393
3 years      116802
N/A          108496
1 year        95926
5 years       88686
4 years       86767
6 years       62261
8 years       51222
7 years       48965
9 years       46393
Name: count, dtype: Int64

In [11]:
##**
raw_df[raw_df['dti'].isnull()]

,application_type,annual_inc,annual_inc_joint,dti,dti_joint,emp_length,issue_d,int_rate,home_ownership,loan_status,loan_amnt,funded_amnt,installment
449,Joint App,0.00,40000.00,NaN,13.65,N/A,Mar-2016,9.16%,MORTGAGE,Fully Paid,2500.00,2500.00,79.69
2224,Joint App,0.00,115000.00,NaN,20.20,N/A,Mar-2016,11.99%,MORTGAGE,Charged Off,12000.00,12000.00,398.52
5419,Joint App,0.00,56000.00,NaN,30.30,N/A,Mar-2016,9.16%,MORTGAGE,Fully Paid,15000.00,15000.00,478.12
10851,Joint App,0.00,65000.00,NaN,8.71,N/A,Mar-2016,12.99%,RENT,Fully Paid,9100.00,9100.00,306.58
16023,Joint App,0.00,50000.00,NaN,9.84,N/A,Mar-2016,22.45%,MORTGAGE,Charged Off,17500.00,17500.00,487.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430415,Joint App,0.00,72000.00,NaN,17.81,N/A,Jan-2019,19.92%,RENT,Current,25000.00,25000.00,661.24
1431977,Joint App,0.00,166442.00,NaN,9.21,N/A,Jan-2019,8.19%,MORTGAGE,Current,10000.00,10000.00,314.25
1432116,Joint App,0.00,103000.00,NaN,18.89,5 years,Jan-2019,7.56%,MORTGAGE,Current,16000.00,16000.00,498.15
1432464,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN


In [12]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432466 entries, 0 to 1432465
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1432466 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   home_ownership    1432440 non-null  string 
 9   loan_status       1432440 non-null  string 
 10  loan_amnt         1432440 non-null  float64
 11  funded_amnt       1432440 non-null  float64
 12  installment       1432440 non-null  float64
dtypes: float64(7), string(6)
memory usage: 142.1 MB


In [13]:
# ตัวอย่างโค้ดเพื่อตรวจสอบว่า ทุกครั้งที่ dti เป็น null แล้ว annual_inc_joint จะมากกว่า 0 เสมอ
condition = (raw_df['dti'].isnull()) & (raw_df['annual_inc_joint'] >= 0)
# คำนวณจำนวนแถวที่ตรงตามเงื่อนไข
matching_rows = raw_df[condition]

# ตรวจสอบว่ามีแถวใดที่ตรงตามเงื่อนไขหรือไม่
if not matching_rows.empty:
    print(f"There are {len(matching_rows)} rows where 'dti' is NULL and 'annual_inc_joint' >= 0.")
else:
    print("No rows match the condition where 'dti' is NULL and 'annual_inc_joint' >= 0.")


There are 1584 rows where 'dti' is NULL and 'annual_inc_joint' >= 0.


In [14]:
# ตัวอย่างโค้ดเพื่อตรวจสอบว่า ทุกครั้งที่ dti เป็น null แล้ว annual_inc_joint จะน้อยกว่าหรือเท่ากับ 0 เสมอ
condition = (raw_df['dti'].isnull()) & (raw_df['annual_inc_joint'] <= 0)
# คำนวณจำนวนแถวที่ตรงตามเงื่อนไข
matching_rows = raw_df[condition]

# ตรวจสอบว่ามีแถวใดที่ตรงตามเงื่อนไขหรือไม่
if not matching_rows.empty:
    print(f"There are {len(matching_rows)} rows where 'dti' is NULL and 'annual_inc_joint' <= 0.")
else:
    print("No rows match the condition where 'dti' is NULL and 'annual_inc_joint' <= 0.")


No rows match the condition where 'dti' is NULL and 'annual_inc_joint' <= 0.


In [15]:
## ไม่น่าจะมีอะไรผิดข้อเท็จจริง เกี่ยวกับ dti ดังนั้น เราจะลองปล่อย dti เป็น null ไปก่อน

In [16]:
# ตัวอย่างโค้ดเพื่อตรวจสอบว่่า ทุกครั้งที่ dti_joint เป็น null แล้ว annual_inc_joint เป็น null ด้วย
condition = (raw_df['dti_joint'].isnull()) & (raw_df['annual_inc_joint'].isnull())
# คำนวณจำนวนแถวที่ตรงตามเงื่อนไข
matching_rows = raw_df[condition]

# แสดงจำนวนแถวที่ตรงตามเงื่อนไข
print(f"Number of rows where both 'dti_joint' and 'annual_inc_joint' are NULL: {len(matching_rows)}")


Number of rows where both 'dti_joint' and 'annual_inc_joint' are NULL: 1320383


In [17]:
matching_rows

,application_type,annual_inc,annual_inc_joint,dti,dti_joint,emp_length,issue_d,int_rate,home_ownership,loan_status,loan_amnt,funded_amnt,installment
0,Individual,53000.00,NaN,20.99,NaN,6 years,Mar-2016,11.99%,MORTGAGE,Fully Paid,12800.00,12800.00,425.09
1,Individual,52000.00,NaN,18.86,NaN,9 years,Mar-2016,15.31%,RENT,Fully Paid,9000.00,9000.00,313.36
2,Individual,73000.00,NaN,10.85,NaN,5 years,Mar-2016,10.75%,RENT,Fully Paid,8000.00,8000.00,260.97
3,Individual,138000.00,NaN,12.44,NaN,10+ years,Mar-2016,12.99%,RENT,Charged Off,25000.00,25000.00,568.70
4,Individual,134000.00,NaN,25.69,NaN,10+ years,Mar-2016,18.25%,MORTGAGE,Fully Paid,35000.00,35000.00,893.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432461,Individual,32000.00,NaN,20.89,NaN,10+ years,Jan-2019,16.14%,MORTGAGE,Current,16000.00,16000.00,563.62
1432462,Individual,72000.00,NaN,7.02,NaN,5 years,Jan-2019,11.31%,MORTGAGE,Current,16000.00,16000.00,350.36
1432463,Individual,65000.00,NaN,29.52,NaN,7 years,Jan-2019,18.94%,MORTGAGE,Current,29250.00,29250.00,757.80
1432464,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN


In [18]:
##จากข้างบนทำให้สงสัยว่า ทุกครั้งที่ application_type = <NA> แล้ว emp_length = N/A และ col. ที่เหลืออื่นๆ เป็น null 

In [19]:
# ตัวอย่างโค้ดในการตรวจสอบเงื่อนไข
condition = (raw_df['application_type'].isnull()) & (raw_df['emp_length'] == 'N/A')

# ลิสต์ของคอลัมน์ที่ต้องการตรวจสอบว่าเป็น null ยกเว้น 'application_type' และ 'emp_length'
columns_to_check = ['annual_inc', 'annual_inc_joint', 'dti', 'dti_joint']

# ตรวจสอบในแถวที่ตรงตามเงื่อนไขของ 'application_type' และ 'emp_length'
filtered_rows = raw_df.loc[condition]

# นับจำนวนแถวที่คอลัมน์อื่นๆ ที่เหลือเป็น null
count_matching_rows = 0
for index, row in filtered_rows.iterrows():
    if row[columns_to_check].isnull().all():  # ตรวจสอบว่าทุกคอลัมน์ใน columns_to_check เป็น null
        count_matching_rows += 1

print(f"Number of rows where 'application_type' is <NA>, 'emp_length' is N/A, and other specified columns are NULL: {count_matching_rows}")

filtered_rows

Number of rows where 'application_type' is <NA>, 'emp_length' is N/A, and other specified columns are NULL: 26


,application_type,annual_inc,annual_inc_joint,dti,dti_joint,emp_length,issue_d,int_rate,home_ownership,loan_status,loan_amnt,funded_amnt,installment
133887,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
133888,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
231743,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
231744,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
330865,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
330866,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
434413,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
434414,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
531194,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
531195,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN


In [20]:
filteredRow_df = raw_df[raw_df['application_type']!='<NA>']

In [21]:
filteredRow_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1432440 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   home_ownership    1432440 non-null  string 
 9   loan_status       1432440 non-null  string 
 10  loan_amnt         1432440 non-null  float64
 11  funded_amnt       1432440 non-null  float64
 12  installment       1432440 non-null  float64
dtypes: float64(7), string(6)
memory usage: 153.0 MB


In [22]:
filteredRow_df.groupby('application_type').size()

application_type
Individual    1320357
Joint App      112083
dtype: int64

In [23]:
##** หากจำเป็นต้องมี annual_inc และ annual_inc_joint ไว้ใน fact table
##   เนื่องจาก การมี annual_inc และ annual_inc_joint ไว้ใน dimension table นั้น
##   จะทำให้เกิด pk ใน dimension table เยอะเกินไป
##   ตรงนี้จึงเป็ฯจุดเริ่มต้นการ วิเคราะห์ และ แสดงให้เห็นว่า เหตุใด จึงควรเก็บ null ไว้ **##

In [24]:
filteredRow_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1432440 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   home_ownership    1432440 non-null  string 
 9   loan_status       1432440 non-null  string 
 10  loan_amnt         1432440 non-null  float64
 11  funded_amnt       1432440 non-null  float64
 12  installment       1432440 non-null  float64
dtypes: float64(7), string(6)
memory usage: 153.0 MB


In [25]:
average_dti_joint = filteredRow_df['dti_joint'].mean()

average_annual_inc_joint = filteredRow_df['annual_inc_joint'].mean()

print(f"Average dti_joint of the co-borrower: {average_dti_joint}")
print(f"Average annual joint income of the co-borrowers: {average_annual_inc_joint}")

Average dti_joint of the co-borrower: 19.08868309526146
Average annual joint income of the co-borrowers: 126843.24936573785


In [26]:
average_dti_joint = filteredRow_df['dti_joint'].fillna(0).mean()
average_annual_inc_joint = filteredRow_df['annual_inc_joint'].fillna(0).mean()

print(f"Average dti_joint of the co-borrower: {average_dti_joint}")
print(f"Average annual joint income of the co-borrowers: {average_annual_inc_joint}")

Average dti_joint of the co-borrower: 1.4935904400882407
Average annual joint income of the co-borrowers: 9925.003433763366


In [27]:
## จะเห็นได้ว่า การแทนที่ null ด้วย 0 ทำให้ค่าเฉลี่ยผิดเพี้ยนไป
## สิ้นสุด การวิเคราะห์ และ แสดงให้เห็นว่า เหตุใด จึงควรเก็บ null ไว้ ##

In [28]:
filteredRow_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1432440 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   home_ownership    1432440 non-null  string 
 9   loan_status       1432440 non-null  string 
 10  loan_amnt         1432440 non-null  float64
 11  funded_amnt       1432440 non-null  float64
 12  installment       1432440 non-null  float64
dtypes: float64(7), string(6)
memory usage: 153.0 MB


In [29]:
## **
# ทำสำเนาตัวแปร โดยหลังจากนี้ หากเกิดความเปลี่ยนแปลงกับ ตัวแปรใหม่ (df_prepared) จะไม่ส่งผลใดๆ ต่อตัวแปรเดิม (noNull_df)
## df_prepared = noNull_df.copy()
df_prepared = filteredRow_df.copy() ##** ณ จุดนี้ filteredRow_df ยังคงมี null แฝงอยู่

# เปลี่ยน data type เป็น datetime สำหรับ col: issue_d
df_prepared['issue_d'] = pd.to_datetime(df_prepared['issue_d'], format='%b-%Y')

# นำเครื่องหมาย % ออกจากค่าใน col: int_rate แล้วเปลี่ยน data type เป็น float
if df_prepared['int_rate'].dtype == 'string':
    df_prepared['int_rate'] = df_prepared['int_rate'].str.rstrip('%').astype('float') / 100.0

In [30]:
df_prepared.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   application_type  1432440 non-null  string        
 1   annual_inc        1432440 non-null  float64       
 2   annual_inc_joint  112083 non-null   float64       
 3   dti               1430856 non-null  float64       
 4   dti_joint         112081 non-null   float64       
 5   emp_length        1432440 non-null  string        
 6   issue_d           1432440 non-null  datetime64[ns]
 7   int_rate          1432440 non-null  float64       
 8   home_ownership    1432440 non-null  string        
 9   loan_status       1432440 non-null  string        
 10  loan_amnt         1432440 non-null  float64       
 11  funded_amnt       1432440 non-null  float64       
 12  installment       1432440 non-null  float64       
dtypes: datetime64[ns](1), float64(8), string(4)
mem

In [31]:
df_prepared.groupby('issue_d').size()

issue_d
2016-01-01     32366
2016-02-01     39529
2016-03-01     61992
2016-04-01     36432
2016-05-01     28403
2016-06-01     33019
2016-07-01     34696
2016-08-01     36280
2016-09-01     28144
2016-10-01     32772
2016-11-01     34591
2016-12-01     36183
2017-01-01    127340
2017-02-01    111052
2017-03-01    148724
2018-01-01     36347
2018-02-01     32746
2018-03-01     38771
2018-04-01     42928
2018-05-01     46311
2018-06-01     41533
2018-07-01     43089
2018-08-01     46079
2018-09-01     39026
2018-10-01     46305
2018-11-01     41973
2018-12-01     40134
2019-01-01     43584
2019-02-01     35985
2019-03-01     36106
dtype: int64

In [32]:
df_prepared.groupby('int_rate').size()

int_rate
0.05     8613
0.05    36953
0.06      493
0.06     5019
0.06     2968
        ...  
0.31     1223
0.31      676
0.31      484
0.31      324
0.31      464
Length: 258, dtype: int64

In [33]:
## dimension table เป็นสิ่งที่อยู่ใน data warehouse ไม่ใช่ python
## แต่เราจะใช้ python ไป create dim. table ใน data warehouse พร้อม insert ข้อมูลลงไปด้วย
## ด้วยเหตุนี้ ครูเอ้จึงเสนอ idea สำหรับการใช้ python พัฒนา dim. table ดังนี้
## (1) สร้าง dataframe ขึ้นมาใหม่ เพื่อทำหน้าที่เป็น dim. table โดยมีเฉพาะ col. ตาม dimensional model ที่ออกแบบไว้แล้ว
## (2) สร้าง col. ใหม่ขึ้นมาบน dataframe ข้อ (1) เพื่อจะใช้เป็น primary key สำหรับ dim. table

# ทำข้อ (1) สำหรับ home_ownership
home_ownership_dim = df_prepared[['home_ownership']].drop_duplicates().reset_index(drop=True)
home_ownership_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ home_ownership
home_ownership_dim['home_ownership_id'] = home_ownership_dim.index

# ทำข้อ (1) สำหรับ loan_status
loan_status_dim = df_prepared[['loan_status']].drop_duplicates().reset_index(drop=True)
loan_status_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ loan_status
loan_status_dim['loan_status_id'] = loan_status_dim.index

# ทำข้อ (1) สำหรับ สำหรับ issue_d
issue_d_dim = df_prepared[['issue_d']].drop_duplicates().reset_index(drop=True)
issue_d_dim['month'] = issue_d_dim['issue_d'].dt.month
issue_d_dim['year'] = issue_d_dim['issue_d'].dt.year
issue_d_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ สำหรับ issue_d
issue_d_dim['issue_d_id'] = issue_d_dim.index

In [34]:
##** เพิ่ม application_type_dim และ emp_length_dim

# สร้าง application_type_dim
application_type_dim = df_prepared[['application_type']].drop_duplicates().reset_index(drop=True)
application_type_dim.reset_index(inplace=True)
application_type_dim['application_type_id'] = application_type_dim.index

# สร้าง emp_length_dim
emp_length_dim = df_prepared[['emp_length']].drop_duplicates().reset_index(drop=True)
emp_length_dim.reset_index(inplace=True)
emp_length_dim['emp_length_id'] = emp_length_dim.index

In [35]:
application_type_dim

,index,application_type,application_type_id
0,0,Individual,0
1,1,Joint App,1


In [36]:
emp_length_dim

,index,emp_length,emp_length_id
0,0,6 years,0
1,1,9 years,1
2,2,5 years,2
3,3,10+ years,3
4,4,8 years,4
5,5,3 years,5
6,6,2 years,6
7,7,< 1 year,7
8,8,4 years,8
9,9,1 year,9


In [37]:
## fact table เป็นสิ่งที่อยู่ใน data warehouse ไม่ใช่ python
## แต่เราจะใช้ python ไป create fact table ใน data warehouse พร้อม insert ข้อมูลลงไปด้วย
## ด้วยเหตุนี้ ครูเอ้จึงเสนอ idea สำหรับการใช้ python พัฒนา fact table ดังนี้
## (1) สร้าง python's dict. ขึ้นมาใช้ key mapping (สร้าง foreign key ของ fact table ตรงกับ primary key ของ dim. table)
## (2) dataframe ขึ้นมาใหม่ เพื่อทำหน้าที่เป็น fact table โดยนำ key mapping ตามข้อ 1 มาใช้ด้วย

In [38]:
# ทำข้อ (1)
home_ownership_map = home_ownership_dim.set_index('home_ownership')['home_ownership_id'].to_dict()
loan_status_map = loan_status_dim.set_index('loan_status')['loan_status_id'].to_dict()
issue_d_map = issue_d_dim.set_index('issue_d')['issue_d_id'].to_dict()

In [39]:
#**
# สร้าง key mapping สำหรับ application_type_dim
application_type_map = application_type_dim.set_index('application_type')['application_type_id'].to_dict()

# สร้าง key mapping สำหรับ emp_length_dim
emp_length_map = emp_length_dim.set_index('emp_length')['emp_length_id'].to_dict()


In [40]:
# ทำข้อ (2)
loans_fact = df_prepared.copy()
loans_fact['home_ownership_id'] = loans_fact['home_ownership'].map(home_ownership_map)
loans_fact['loan_status_id'] = loans_fact['loan_status'].map(loan_status_map)
loans_fact['issue_d_id'] = loans_fact['issue_d'].map(issue_d_map)

#**
# เชื่อมค่า application_type ใน loans_fact กับ application_type_map
loans_fact['application_type_id'] = loans_fact['application_type'].map(application_type_map)

# เชื่อมค่า emp_length ใน loans_fact กับ emp_length_map
loans_fact['emp_length_id'] = loans_fact['emp_length'].map(emp_length_map)


# เลือกคอลัมน์ที่จำเป็นสำหรับ Fact Table
loans_fact = loans_fact[['application_type','loan_amnt', 'funded_amnt', 'int_rate', 'installment'\
                         ,'home_ownership_id', 'loan_status_id', 'issue_d_id'\
                         ,'application_type_id','emp_length_id'\
                         ,'annual_inc','annual_inc_joint','dti','dti_joint']]

In [41]:
loans_fact.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 14 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   application_type     1432440 non-null  string 
 1   loan_amnt            1432440 non-null  float64
 2   funded_amnt          1432440 non-null  float64
 3   int_rate             1432440 non-null  float64
 4   installment          1432440 non-null  float64
 5   home_ownership_id    1432440 non-null  int64  
 6   loan_status_id       1432440 non-null  int64  
 7   issue_d_id           1432440 non-null  int64  
 8   application_type_id  1432440 non-null  int64  
 9   emp_length_id        1432440 non-null  int64  
 10  annual_inc           1432440 non-null  float64
 11  annual_inc_joint     112083 non-null   float64
 12  dti                  1430856 non-null  float64
 13  dti_joint            112081 non-null   float64
dtypes: float64(8), int64(5), string(1)
memory usage: 163.9 

In [42]:
df_prepared.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   application_type  1432440 non-null  string        
 1   annual_inc        1432440 non-null  float64       
 2   annual_inc_joint  112083 non-null   float64       
 3   dti               1430856 non-null  float64       
 4   dti_joint         112081 non-null   float64       
 5   emp_length        1432440 non-null  string        
 6   issue_d           1432440 non-null  datetime64[ns]
 7   int_rate          1432440 non-null  float64       
 8   home_ownership    1432440 non-null  string        
 9   loan_status       1432440 non-null  string        
 10  loan_amnt         1432440 non-null  float64       
 11  funded_amnt       1432440 non-null  float64       
 12  installment       1432440 non-null  float64       
dtypes: datetime64[ns](1), float64(8), string(4)
mem

In [43]:
### เริ่มต้นเปลี่ยนเป็น raw cell ชั่วคราว ###

In [44]:
## [optional] ทดสอบการ join ทุก dataframe ทั้ง fact และ dim. เข้าด้วยกัน

# Join `loans_fact` กับ `home_ownership_dim` โดยใช้ 'home_ownership_id'
loans_fact_with_home_ownership = pd.merge(loans_fact, home_ownership_dim, on='home_ownership_id', how='left', suffixes=('', '_home_ownership'))

# Join ผลลัพธ์กับ `loan_status_dim` โดยใช้ 'loan_status_id'
loans_fact_with_home_ownership_with_loan_status = pd.merge(loans_fact_with_home_ownership, loan_status_dim, on='loan_status_id', how='left', suffixes=('', '_loan_status'))

# Join ผลลัพธ์กับ `issue_d_dim` โดยใช้ 'issue_d_id'
final_df = pd.merge(loans_fact_with_home_ownership_with_loan_status, issue_d_dim, on='issue_d_id', how='left', suffixes=('', '_issue_d'))


In [45]:
##**

# เพิ่ม application_type_dim โดยใช้ 'application_type_id'
final_df_with_application_type = pd.merge(final_df, application_type_dim, on='application_type_id', how='left', suffixes=('', '_application_type'))

# เพิ่ม emp_length_dim โดยใช้ 'emp_length_id'
final_df_with_all_dims = pd.merge(final_df_with_application_type, emp_length_dim, on='emp_length_id', how='left', suffixes=('', '_emp_length'))

# final_df_with_all_dims ตอนนี้มีการ join กับทุก dimensions รวมถึง application_type_dim และ emp_length_dim

In [46]:
##**
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องมี row เท่ากันกับ dataframe เดิม
## print(f"จำนวนแถวใน noNull_df: {noNull_df.shape[0]}")
print(f"จำนวนแถวใน filteredRow_df: {filteredRow_df.shape[0]}")
print(f"จำนวนแถวใน final_df: {final_df.shape[0]}")

จำนวนแถวใน filteredRow_df: 1432440
จำนวนแถวใน final_df: 1432440


In [47]:
##**
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องมีค่าของ measure เท่ากันกับ dataframe เดิม
print("การเปรียบเทียบค่า loan_amnt จาก dataframe เดิม กับ fact:")
print(filteredRow_df['funded_amnt'].head())
##print(noNull_df['funded_amnt'].head())
print(final_df['funded_amnt'].head())


การเปรียบเทียบค่า loan_amnt จาก dataframe เดิม กับ fact:
0   12800.00
1    9000.00
2    8000.00
3   25000.00
4   35000.00
Name: funded_amnt, dtype: float64
0   12800.00
1    9000.00
2    8000.00
3   25000.00
4   35000.00
Name: funded_amnt, dtype: float64


In [48]:
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องไม่มี NULL เลย
print("จำนวนค่า Null ใน final_df หลังจากการ join:")
print(final_df.isnull().sum())


จำนวนค่า Null ใน final_df หลังจากการ join:
application_type             0
loan_amnt                    0
funded_amnt                  0
int_rate                     0
installment                  0
home_ownership_id            0
loan_status_id               0
issue_d_id                   0
application_type_id          0
emp_length_id                0
annual_inc                   0
annual_inc_joint       1320357
dti                       1584
dti_joint              1320359
index                        0
home_ownership               0
index_loan_status            0
loan_status                  0
index_issue_d                0
issue_d                      0
month                        0
year                         0
dtype: int64


In [49]:
### สิ้นสุดเปลี่ยนเป็น raw cell ชั่วคราว ###

In [50]:
server = '34.136.225.236'
database = 'loanDW'
username = 'SA'
password = 'Passw0rd123456'

In [51]:
issue_d_dim.drop(columns=['index'], inplace=True)

In [52]:
loan_status_dim.drop(columns=['index'], inplace=True)

In [53]:
home_ownership_dim.drop(columns=['index'], inplace=True)

In [54]:
from sqlalchemy import create_engine
import urllib

# ตั้งค่าการเชื่อมต่อกับ MSSQL โดยใช้ข้อมูลที่เหมาะสมกับสภาพแวดล้อมของคุณ
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=server;DATABASE=database;UID=username;PWD=password")
##### engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
# Using pymssql
engine = create_engine(f'mssql+pymssql://{username}:{password}@{server}/{database}')


# นำเข้าข้อมูล Dimension Tables ไปยัง MSSQL ให้ตรงกับขั้นตอนการเตรียมข้อมูลที่คุณทำไว้
##### home_ownership_dim.to_sql('home_ownership_dim', con=engine, if_exists='replace', index=True, index_label='home_ownership_id')
##### loan_status_dim.to_sql('loan_status_dim', con=engine, if_exists='replace', index=True, index_label='loan_status_id')

home_ownership_dim.to_sql('home_ownership_dim', con=engine, if_exists='replace', index=False)
loan_status_dim.to_sql('loan_status_dim', con=engine, if_exists='replace', index=False)
issue_d_dim.to_sql('issue_d_dim', con=engine, if_exists='replace', index=False)



# นำเข้าข้อมูล Fact Table ไปยัง MSSQL โดยใช้ข้อมูลที่เตรียมไว้
loans_fact.to_sql('loans_fact', con=engine, if_exists='replace', index=False)


103

In [55]:
application_type_dim.to_sql('application_type_dim', con=engine, if_exists='replace', index=False)
emp_length_dim.to_sql('emp_length_dim', con=engine, if_exists='replace', index=False)

12